# Cuisine Classification from Ingredients

### **Description:** The main purpose of this project is to classify cuisines based on Ingredients of the Recipe.


The steps followed are as follows:
* Step-1: Download dataset from [https://www.kaggle.com/c/whats-cooking/data](https://www.kaggle.com/c/whats-cooking/data), store it in dictionary and convert it to dataframe.
* Step-2: Feature Selection:
  * Removal of punctuation, digits, content inside parenthesis using Regex Expression
  * Remove brand names using Regex Expression
  * Convert to lower case and Remove stop words using Regex Expression
  * Use Porter Stemmer Algorithm
* Step-3: Encoded Cuisine column using Label Encoder of sklearn.
* Step-4: Convert the ingredients column after feature selection into TFIDF Matrix
* Step-5: Split the data(X-TFIDF Matrix, Y-Label Encoded value of Cuisine into training and test data(80:20).
* Step-6: Use Different Machine Learning Algorithm to get best accuracy. 


June 24 Update: this was forked from Kaggle, https://www.kaggle.com/rahulsridhar2811/cuisine-classification-with-accuracy-78-88
Still working on improving it, and possibly applying ULMFiT to this dataset for higher accuracy

In [1]:
import json
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

ps = PorterStemmer()
file = r'/Users/xinrucheng/Documents/Projects/recsys-recipes/recipe-ingredients-dataset/train.json' #relative path?
with open(file) as train_file:
    dict_train = json.load(train_file)

print(dict_train[0])

{'id': 10259, 'cuisine': 'greek', 'ingredients': ['romaine lettuce', 'black olives', 'grape tomatoes', 'garlic', 'pepper', 'purple onion', 'seasoning', 'garbanzo beans', 'feta cheese crumbles']}


In [2]:
len(dict_train)

39774

In [3]:
dict_train[0]['ingredients'] #print ingredients of the first recipe

['romaine lettuce',
 'black olives',
 'grape tomatoes',
 'garlic',
 'pepper',
 'purple onion',
 'seasoning',
 'garbanzo beans',
 'feta cheese crumbles']

In [4]:
id_ = []
cuisine = []
ingredients = []
for i in range(len(dict_train)):
    id_.append(dict_train[i]['id'])
    cuisine.append(dict_train[i]['cuisine'])
    ingredients.append(dict_train[i]['ingredients'])
    
    #loop through items, separate into categories to build df
    

In [5]:
import pandas as pd
df = pd.DataFrame({'id':id_, 
                   'cuisine':cuisine, 
                   'ingredients':ingredients})
print(df.head(5)) #info of first five recipes

       cuisine     id                                        ingredients
0        greek  10259  [romaine lettuce, black olives, grape tomatoes...
1  southern_us  25693  [plain flour, ground pepper, salt, tomatoes, g...
2     filipino  20130  [eggs, pepper, salt, mayonaise, cooking oil, g...
3       indian  22213                [water, vegetable oil, wheat, salt]
4       indian  13162  [black pepper, shallots, cornflour, cayenne pe...


In [7]:
df['cuisine'].value_counts()

italian         7838
mexican         6438
southern_us     4320
indian          3003
chinese         2673
french          2646
cajun_creole    1546
thai            1539
japanese        1423
greek           1175
spanish          989
korean           830
vietnamese       825
moroccan         821
british          804
filipino         755
irish            667
jamaican         526
russian          489
brazilian        467
Name: cuisine, dtype: int64

In [9]:
new = []
for s in df['ingredients']:
    s = ' '.join(s) #**review pandas doc: .join - Join columns of another DataFrame (not how it's used here)
    new.append(s)#join ingredients with spaces instead, save as new df in the cell below
    

In [10]:
df['ing'] = new

In [11]:
import re #re?
l=[]
for s in df['ing']:
    
    #Remove punctuations
    s=re.sub(r'[^\w\s]','',s)
    
    #Remove Digits
    s=re.sub(r"(\d)", "", s)
    
    #Remove content inside parenthesis
    s=re.sub(r'\([^)]*\)', '', s)
    
    #Remove Brand Name
    s=re.sub(u'\w*\u2122', '', s)
    
    #Convert to lowercase
    s=s.lower()
    
    #Remove Stop Words
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(s)
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    filtered_sentence = []
    for w in word_tokens:
        if w not in stop_words:
            filtered_sentence.append(w)
    s=' '.join(filtered_sentence) #join the non-stopwords with spaces
    
    #Remove low-content adjectives
    
    
    #Porter Stemmer Algorithm
    words = word_tokenize(s)
    word_ps=[]
    for w in words:
       word_ps.append(ps.stem(w))
    s=' '.join(word_ps)
    
    l.append(s)
df['ing_mod']=l
print(df.head(10)) #doesn't seem to work well? removed plural forms which is good, but also removed some ending e's (and vegetable becomes veget)

       cuisine     id                                        ingredients  \
0        greek  10259  [romaine lettuce, black olives, grape tomatoes...   
1  southern_us  25693  [plain flour, ground pepper, salt, tomatoes, g...   
2     filipino  20130  [eggs, pepper, salt, mayonaise, cooking oil, g...   
3       indian  22213                [water, vegetable oil, wheat, salt]   
4       indian  13162  [black pepper, shallots, cornflour, cayenne pe...   
5     jamaican   6602  [plain flour, sugar, butter, eggs, fresh ginge...   
6      spanish  42779  [olive oil, salt, medium shrimp, pepper, garli...   
7      italian   3735  [sugar, pistachio nuts, white almond bark, flo...   
8      mexican  16903  [olive oil, purple onion, fresh pineapple, por...   
9      italian  12734  [chopped tomatoes, fresh basil, garlic, extra-...   

                                                 ing  \
0  romaine lettuce black olives grape tomatoes ga...   
1  plain flour ground pepper salt tomatoes ground..

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer #**look up this vectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['ing_mod'])

print(X)
#print(vectorizer.get_feature_names())

  (0, 2038)	0.358742368305
  (0, 1350)	0.279113558822
  (0, 207)	0.1474807692
  (0, 1675)	0.140383928524
  (0, 1026)	0.358370896832
  (0, 2454)	0.149020782233
  (0, 953)	0.110423617226
  (0, 1788)	0.104595496092
  (0, 1936)	0.250742183159
  (0, 1678)	0.112545141153
  (0, 2129)	0.235344428308
  (0, 948)	0.406987699414
  (0, 151)	0.209263619966
  (0, 839)	0.318797276253
  (0, 457)	0.152756710941
  (0, 645)	0.335930359104
  (1, 207)	0.170560662478
  (1, 2454)	0.34468335741
  (1, 1788)	0.241928180905
  (1, 1853)	0.345327620662
  (1, 884)	0.184355427894
  (1, 1053)	0.302640570155
  (1, 2087)	0.108657345321
  (1, 2438)	0.273543336047
  (1, 778)	0.184685524469
  :	:
  (39772, 2326)	0.268411022775
  (39772, 248)	0.268411022775
  (39772, 2309)	0.248396991662
  (39772, 2578)	0.249620356348
  (39773, 207)	0.159684154764
  (39773, 2454)	0.161351597108
  (39773, 953)	0.119560686307
  (39773, 1788)	0.22650062752
  (39773, 1678)	0.121857757017
  (39773, 1053)	0.141670719791
  (39773, 2087)	0.10172835

Note (scikit-learn doc): Feature extraction is very different from Feature selection: the former consists in transforming arbitrary data, such as text or images, into numerical features usable for machine learning. The latter is a machine learning technique applied on these features.

In [13]:
len(new)

39774

In [14]:
type(df['ing'][0])

str

In [15]:
s='1 1cool co1l coo1'
s=re.sub(r"(\d)", "", s) #check remove digits
print(s) 

 cool col coo


In [18]:
s='hi 1(bye)'
s=re.sub(r'\([^)]*\)', '', s)     #Remove content inside parenthesis

print(s)

hi 1


In [19]:
s='hi 1 Marvel™ hi'
s=re.sub(u'\w*\u2122', '', s)  #remove brand labels
print(s)

hi 1  hi


In [33]:
import re
from nltk.corpus import stopwords
s="I love this phone, its super fast and there's so much new and cool things with jelly bean....but of recently I've seen some bugs."
pattern = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*')
s = pattern.sub('', s)
print(s)

I love phone, super fast 'much new cool things jelly bean....recently I'seen bugs.


In [26]:
#"love phone, super fast much cool jelly bean....but recently bugs."
#import nltk
#nltk.download('word_tokenize') #need to search for nltk package?** but next cell works when importing package

In [34]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

example_sent = "This is a sample sentence, showing off the stop words filtration."

stop_words = set(stopwords.words('english'))

word_tokens = word_tokenize(example_sent)

filtered_sentence = [w for w in word_tokens if not w in stop_words]

filtered_sentence = []

for w in word_tokens:
    if w not in stop_words:
        filtered_sentence.append(w)

print(word_tokens)
print(filtered_sentence)

['This', 'is', 'a', 'sample', 'sentence', ',', 'showing', 'off', 'the', 'stop', 'words', 'filtration', '.']
['This', 'sample', 'sentence', ',', 'showing', 'stop', 'words', 'filtration', '.']


In [27]:
#"love phone, super fast much cool jelly bean....but recently bugs."

In [28]:
#import nltk
#nltk.download('word_tokenize') #2nd time?**

In [25]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/xinrucheng/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [36]:
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

ps = PorterStemmer()

In [42]:
new_text = "It is important to be very pythonly while you are pythoning with python. All pythoners have pythoned poorly at least once."

#print(s)#?
print(new_text)

It is important to be very pythonly while you are pythoning with python. All pythoners have pythoned poorly at least once.


In [43]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize  #duplicate** delete?

#s = "This is a sample sentence, showing off the stop words filtration."

stop_words = set(stopwords.words('english'))

word_tokens = word_tokenize(new_text)

filtered_sentence = [w for w in word_tokens if not w in stop_words]

filtered_sentence = []

for w in word_tokens:
    if w not in stop_words:
        filtered_sentence.append(w)

print(word_tokens)
print(filtered_sentence) #haven't applied ps yet**

['It', 'is', 'important', 'to', 'be', 'very', 'pythonly', 'while', 'you', 'are', 'pythoning', 'with', 'python', '.', 'All', 'pythoners', 'have', 'pythoned', 'poorly', 'at', 'least', 'once', '.']
['It', 'important', 'pythonly', 'pythoning', 'python', '.', 'All', 'pythoners', 'pythoned', 'poorly', 'least', '.']


In [44]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(df['cuisine'])
df['cuisine']=le.transform(df['cuisine']) 

In [47]:
df['cuisine'].value_counts() 
#now showing how many recipes belong to each cuisine type, where cuisine type is labelled as numbers

9     7838
13    6438
16    4320
7     3003
3     2673
5     2646
2     1546
18    1539
11    1423
6     1175
17     989
12     830
19     825
14     821
1      804
4      755
8      667
10     526
15     489
0      467
Name: cuisine, dtype: int64

In [48]:
cuisine_map={'0':'brazilian', '1':'british', '2':'cajun_creole', '3':'chinese', '4':'filipino', '5':'french', '6':'greek', '7':'indian', '8':'irish', '9':'italian', '10':'jamaican', '11':'japanese', '12':'korean', '13':'mexican', '14':'moroccan', '15':'russian', '16':'southern_us', '17':'spanish', '18':'thai', '19':'vietnamese'}

In [49]:
Y=[]
Y = df['cuisine']


In [50]:
import numpy as np
from sklearn.preprocessing import Imputer
from sklearn.cross_validation import train_test_split #get DeprecationWarning - change to newer module: model_selection?
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

/Library/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [51]:
import numpy as np
from sklearn.preprocessing import Imputer
from sklearn.model_selection import train_test_split #get DeprecationWarning above - replaced with newer module: model_selection
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [52]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 100) #train-test 80-20 split

In [53]:
for K in range(25):
 K_value = K+1
 neigh = KNeighborsClassifier(n_neighbors = K_value, weights='uniform', algorithm='auto')
 neigh.fit(X_train, y_train) 
 y_pred = neigh.predict(X_test)
 print("Accuracy is ", accuracy_score(y_test,y_pred)*100,"% for K-Value:",K_value)


Accuracy is  70.1822752986 % for K-Value: 1
Accuracy is  68.409805154 % for K-Value: 2
Accuracy is  71.690760528 % for K-Value: 3
Accuracy is  72.8472658705 % for K-Value: 4
Accuracy is  73.3249528598 % for K-Value: 5
Accuracy is  73.9912005028 % for K-Value: 6
Accuracy is  74.1923318668 % for K-Value: 7
Accuracy is  74.5191703331 % for K-Value: 8
Accuracy is  74.6951602766 % for K-Value: 9
Accuracy is  74.4688874921 % for K-Value: 10
Accuracy is  74.7831552483 % for K-Value: 11
Accuracy is  74.7077309868 % for K-Value: 12
Accuracy is  74.5694531741 % for K-Value: 13
Accuracy is  74.4688874921 % for K-Value: 14
Accuracy is  74.7328724073 % for K-Value: 15
Accuracy is  74.720301697 % for K-Value: 16
Accuracy is  74.770584538 % for K-Value: 17
Accuracy is  74.6825895663 % for K-Value: 18
Accuracy is  74.4940289126 % for K-Value: 19
Accuracy is  74.5568824639 % for K-Value: 20
Accuracy is  74.4688874921 % for K-Value: 21
Accuracy is  74.2677561282 % for K-Value: 22
Accuracy is  74.2928975

In [54]:
#Implement KNN(So we take K value to be 11, becuase it gives the highest accuracy) 
#(look up KNN, why range 25 in previous cell?)**
neigh = KNeighborsClassifier(n_neighbors = 11, weights='uniform', algorithm='auto')
neigh.fit(X_train, y_train) 
y_pred = neigh.predict(X_test)
print("Accuracy is ", accuracy_score(y_test,y_pred)*100,"% for K-Value:11")

Accuracy is  74.7831552483 % for K-Value:11


In [57]:
#Implement Grid Serch for best Gamma, C and Selection between rbf and linear kernel **look up OVA, Grid Search
from sklearn import svm, datasets
#from sklearn.cross_validation import StratifiedKFold
from sklearn.model_selection import StratifiedKFold
#from sklearn.grid_search import GridSearchCV #another DeprecationWarning, replaced with model_selection
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
parameter_candidates = [
  {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
  {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
]
clf = GridSearchCV(estimator=svm.SVC(), param_grid=parameter_candidates, n_jobs=-1)
clf.fit(X_train, y_train)   
print('Best score for data1:', clf.best_score_) 
print('Best C:',clf.best_estimator_.C) 
print('Best Kernel:',clf.best_estimator_.kernel)
print('Best Gamma:',clf.best_estimator_.gamma)

Best score for data1: 0.776234325403
Best C: 1
Best Kernel: linear
Best Gamma: auto


In [58]:
#OVA SVM(Grid Search Results: Kernel - Linear, C - 1, Gamma - Auto)
from sklearn import svm
lin_clf = svm.LinearSVC(C=1)
lin_clf.fit(X_train, y_train)
y_pred=lin_clf.predict(X_test)
print(accuracy_score(y_test,y_pred)*100)

78.8812067882


In [59]:
#SVM by Crammer(Grid Search Results: Gamma - , C - )
lin_clf = svm.LinearSVC(C=1.0, multi_class='crammer_singer')
lin_clf.fit(X_train, y_train)
y_pred=lin_clf.predict(X_test)
print(accuracy_score(y_test,y_pred)*100)

78.8057825267


In [60]:
#Implementing OVA Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(accuracy_score(y_test,y_pred)*100)

66.7756128221


In [61]:
#Implementing OVA Logistic Regerssion
from sklearn.linear_model import LogisticRegression
logisticRegr = LogisticRegression()
logisticRegr.fit(X_train, y_train)
y_pred = logisticRegr.predict(X_test)
print(accuracy_score(y_test,y_pred)*100)

77.7372721559


In [62]:
df3=pd.DataFrame({'id':y_test.index, 'cuisine':y_test.values})
y_test2=[]
y_test1=df3['cuisine'].tolist()
for i in range(len(df3['cuisine'])):
    y_test2.append(cuisine_map[str(df3['cuisine'][i])])
print(y_test2)

['mexican', 'southern_us', 'italian', 'mexican', 'japanese', 'mexican', 'italian', 'indian', 'italian', 'moroccan', 'french', 'mexican', 'japanese', 'southern_us', 'italian', 'mexican', 'italian', 'japanese', 'indian', 'japanese', 'mexican', 'chinese', 'mexican', 'spanish', 'italian', 'greek', 'vietnamese', 'italian', 'thai', 'jamaican', 'italian', 'greek', 'cajun_creole', 'french', 'japanese', 'italian', 'cajun_creole', 'french', 'british', 'british', 'italian', 'italian', 'italian', 'irish', 'mexican', 'moroccan', 'indian', 'french', 'mexican', 'mexican', 'mexican', 'southern_us', 'mexican', 'southern_us', 'chinese', 'southern_us', 'mexican', 'japanese', 'greek', 'brazilian', 'southern_us', 'italian', 'indian', 'italian', 'chinese', 'mexican', 'vietnamese', 'french', 'jamaican', 'italian', 'mexican', 'italian', 'chinese', 'filipino', 'italian', 'chinese', 'indian', 'indian', 'italian', 'vietnamese', 'greek', 'greek', 'chinese', 'russian', 'italian', 'indian', 'mexican', 'filipino', '

In [63]:
#Convert Predicted Output 
#_______ gives the best accuracy. So lets implement it on last time to get the final output.
y_pred1=[]
for i in range(len(y_pred)):
    y_pred1.append(cuisine_map[str(y_pred[i])])
print(y_test2)

['mexican', 'southern_us', 'italian', 'mexican', 'japanese', 'mexican', 'italian', 'indian', 'italian', 'moroccan', 'french', 'mexican', 'japanese', 'southern_us', 'italian', 'mexican', 'italian', 'japanese', 'indian', 'japanese', 'mexican', 'chinese', 'mexican', 'spanish', 'italian', 'greek', 'vietnamese', 'italian', 'thai', 'jamaican', 'italian', 'greek', 'cajun_creole', 'french', 'japanese', 'italian', 'cajun_creole', 'french', 'british', 'british', 'italian', 'italian', 'italian', 'irish', 'mexican', 'moroccan', 'indian', 'french', 'mexican', 'mexican', 'mexican', 'southern_us', 'mexican', 'southern_us', 'chinese', 'southern_us', 'mexican', 'japanese', 'greek', 'brazilian', 'southern_us', 'italian', 'indian', 'italian', 'chinese', 'mexican', 'vietnamese', 'french', 'jamaican', 'italian', 'mexican', 'italian', 'chinese', 'filipino', 'italian', 'chinese', 'indian', 'indian', 'italian', 'vietnamese', 'greek', 'greek', 'chinese', 'russian', 'italian', 'indian', 'mexican', 'filipino', '

In [64]:
#We Choose OVA SVM as it gives the best accuracy of 78.88%
from sklearn import svm
lin_clf = svm.LinearSVC(C=1)
lin_clf.fit(X_train, y_train)
y_pred=lin_clf.predict(X_test)
print(accuracy_score(y_test,y_pred)*100)
result=pd.DataFrame({'Actual Cuisine':y_test2, 'Predicted Cuisine':y_test2})
print(result)

78.8812067882
     Actual Cuisine Predicted Cuisine
0           mexican           mexican
1       southern_us       southern_us
2           italian           italian
3           mexican           mexican
4          japanese          japanese
5           mexican           mexican
6           italian           italian
7            indian            indian
8           italian           italian
9          moroccan          moroccan
10           french            french
11          mexican           mexican
12         japanese          japanese
13      southern_us       southern_us
14          italian           italian
15          mexican           mexican
16          italian           italian
17         japanese          japanese
18           indian            indian
19         japanese          japanese
20          mexican           mexican
21          chinese           chinese
22          mexican           mexican
23          spanish           spanish
24          italian           italia

# Conclusion
The OVA SVM Algorithm gives the best accuracy of 78.88% for the given dataset.

|       Algorithm       |           Parameters           | Accuracy |
|:----------------------|:------------------------------:|:--------:|
|          KNN          |              K=11              |  74.78%  |
|        OVA SVM        |              C=1               |  78.88%  |
|      Crammer SVM      |C=1, multi_class=crammer_singer |  78.84%  |
|    OVA Naive Bayes    |               -                |  66.71%  |
|OVA Logistic Regression|               -                |  77.72%  |


In [ ]:
#table above, acutally used K=11 for highest KNN accuracy (fixed inconsistency in original table)